## MOEA

In [3]:
from model_with_policy import borg_ebola

import numpy as np

from ema_workbench import (Model, IntegerParameter, RealParameter, ReplicatorModel,
                           TimeSeriesOutcome, ScalarOutcome, ArrayOutcome, Policy, Scenario)

#model = Model('Ebola', function=borg_ebola)
model = ReplicatorModel('Ebola', function=borg_ebola)

model.replications = 5

model.outcomes = [ScalarOutcome('mean effectiveness', ScalarOutcome.MINIMIZE, function=np.mean, 
                               variable_name='Effectiveness'),
                  ScalarOutcome('mean time until containment', ScalarOutcome.MINIMIZE, function = np.mean, 
                               variable_name='Time until Containment'),
                  ScalarOutcome('mean difference in met demand', ScalarOutcome.MINIMIZE, function = np.mean, 
                               variable_name='Difference in Met Demand'),
                  ScalarOutcome('mean difference in arrival time', ScalarOutcome.MINIMIZE, function = np.mean, 
                               variable_name='Difference in Arrival Time'),
                  ScalarOutcome('mean cost per death prevented', ScalarOutcome.MINIMIZE, function = np.mean, 
                               variable_name='Cost per Death Prevented')]

            
model.uncertainties = [IntegerParameter('I4', 1, 8),
                       IntegerParameter('I14', 20, 35),
                       IntegerParameter('I15', 25, 40),
                       RealParameter('beta_i', 0.1, 0.5),
                       RealParameter('travel_rate', 0.04, 0.1)]


model.levers = [RealParameter('c1', -1.0, 1.0),
               RealParameter('c2', -1.0, 1.0),
               RealParameter('r1', 0.000001, 1.0),
               RealParameter('r2', 0.000001, 1.0),
               RealParameter('w', 0, 1.0)]
                

reference = Scenario('reference', I4 =3, I14 = 25, I15 = 32, beta_i = 0.32, travel_rate = 0.05)  

In [4]:
from ema_workbench import SequentialEvaluator, MultiprocessingEvaluator, ema_logging

from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)

ema_logging.log_to_stderr(ema_logging.INFO)

convergence_metrics = [HyperVolume(minimum=[-1,0,0,0,0], maximum=[0,26, 3, 2000, 1000000]),
                       EpsilonProgress()]


#with SequentialEvaluator(model) as evaluator:
with MultiprocessingEvaluator(model) as evaluator:
    results, convergence = evaluator.optimize(nfe=5, searchover='levers',
                                 convergence=convergence_metrics,
                                 epsilons=[0.02, 2, 0.02, 20, 100],
                                 reference=reference)
    
    
    
import pandas as pd
results.to_csv('optimization_results')
convergence.to_csv('convergence')

[MainProcess/INFO] pool started
[MainProcess/INFO] generation 0: 0/5 nfe
[MainProcess/INFO] optimization completed, found 19 solutions
[MainProcess/INFO] terminating pool


In [5]:
print(results)
print(convergence)

          c1        c2        r1        r2         w  mean effectiveness  \
0  -0.778236  0.993524  0.830340  0.302128  0.356558           -0.532122   
1   0.821742  0.851250  0.924757  0.975154  0.690200           -0.834594   
2   0.424489  0.599351  0.968607  0.144504  0.074415           -0.753625   
3  -0.287597 -0.624272  0.430219  0.453996  0.415622           -0.420876   
4   0.160214  0.605888  0.944283  0.177887  0.182978           -0.749032   
5   0.368609  0.664853  0.277612  0.444235  0.435190           -0.657079   
6  -0.095870 -0.514715  0.170478  0.240287  0.728119           -0.465617   
7   0.921653  0.376972  0.689541  0.618193  0.296660           -0.829265   
8  -0.238029  0.390873  0.061404  0.658844  0.354198           -0.394910   
9  -0.684508  0.594549  0.944961  0.839558  0.090236           -0.814826   
10  0.550667  0.829680  0.328605  0.737477  0.162422           -0.837041   
11  0.610319  0.701386  0.345260  0.649909  0.908176           -0.800954   
12  0.317986